In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [4]:
df.describe()

,artist,song,link,text
count,57650,57650,57650,57650
unique,643,44824,57650,57494
top,Donna Summer,Have Yourself A Merry Little Christmas,/z/zwan/heartsong_20148991.html,I just came back from a lovely trip along the ...
freq,191,35,1,6


In [5]:
df.drop_duplicates(inplace = True)

In [7]:
df = df.sample(5000).drop(['link'] , axis = 1).reset_index(drop = True)

In [8]:
df

,artist,song,text
0,INXS,Original Sin,You might know of the original sin \r\nAnd yo...
1,Elton John,It's Getting Dark In Here,It's getting dark in here \r\nDon't want to l...
2,Depeche Mode,Always,The well is dry and the streams runs shallow ...
3,Kris Kristofferson,From The Bottle To The Bottom,You ask me if I'm happy now \r\nThat's good a...
4,Neil Diamond,Blue Destiny (Version One),How can I love \r\nIf my love doesn't love on...
...,...,...,...
4995,Warren Zevon,Bad Karma,Was it something I did \r\nIn another life? ...
4996,Widespread Panic,Use Me,My friends \r\nFeel its their appointed duty ...
4997,Hank Williams Jr.,Tee Tot Song,Every mornin' ole Tee Tot come 'round \r\nSet...
4998,John Denver,Love Again,I didn't think it could happen again \r\nI'm ...


In [9]:
df.shape

(5000, 3)

In [10]:
# cleaning text column 

df['text'] = df['text'].str.lower().replace(r'^a-zA-Z0-9' , ' ').replace(r'^\n' , " " , regex = True)

In [11]:
df['text']

0       you might know of the original sin  \r\nand yo...
1       it's getting dark in here  \r\ndon't want to l...
2       the well is dry and the streams runs shallow  ...
3       you ask me if i'm happy now  \r\nthat's good a...
4       how can i love  \r\nif my love doesn't love on...
                              ...                        
4995    was it something i did  \r\nin another life?  ...
4996    my friends  \r\nfeel its their appointed duty ...
4997    every mornin' ole tee tot come 'round  \r\nset...
4998    i didn't think it could happen again  \r\ni'm ...
4999    oh the prairie lights are burnin' bright  \r\n...
Name: text, Length: 5000, dtype: object

In [27]:
import nltk 

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stems = PorterStemmer()

lemms = WordNetLemmatizer()

In [25]:
def token(txt): 
    for i in txt: 
        token = nltk.word_tokenize(txt)
        a = [lemms.lemmatize(word) for word in token]
        return ' '.join(a)

In [26]:
token("you are beautiful , beauty")

'you are beautiful , beauty'

In [31]:
df['text'].apply(lambda x : token(x))

0       you might know of the origin sin and you might...
1       it 's get dark in here do n't want to leav sha...
2       the well is dri and the stream run shallow the...
3       you ask me if i 'm happi now that 's good as a...
4       how can i love if my love doe n't love onli me...
                              ...                        
4995    wa it someth i did in anoth life ? i tri and t...
4996    my friend feel it their appoint duti they keep...
4997    everi mornin ' ole tee tot come 'round settin'...
4998    i did n't think it could happen again i 'm jus...
4999    oh the prairi light are burnin ' bright the ch...
Name: text, Length: 5000, dtype: object

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

idf = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = idf.fit_transform(df['text'])

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(matrix)

In [35]:
df[df['song'] =="Bad Karma"].index[0]

np.int64(4995)

In [45]:
def recommend(song_name): 
    idx = df[df['song'] ==song_name].index[0]
    distance = sorted(list(enumerate(similarity[idx])),reverse = True , key = lambda x:x[1])

    song = []

    for song_id in distance[1:5]:
        song.append(df.iloc[song_id[0]].song)
    return song

In [48]:
recommend('Roxanne')

['Red Lenses', 'Red Lights', 'Red', 'Red Rag Top']

In [50]:
import pickle

pickle.dump(similarity , open("similarity" , 'wb'))

